In [1]:
!pip install -q -U zarr
!pip install -q -U fsspec
!pip install -q -U gcsfs
!pip install -q -U xarray
!pip install -q -U dask-ml[complete]  
!wget -q https://raw.githubusercontent.com/Competencia-de-Climate-Change/WeatherBench/master/src/ml_training.py

     |████████████████████████████████| 3.3MB 3.3MB/s 
     |████████████████████████████████| 3.8MB 46.8MB/s 
     |████████████████████████████████| 706kB 3.4MB/s 
     |████████████████████████████████| 143kB 3.3MB/s 
     |████████████████████████████████| 645kB 11.1MB/s 
     |████████████████████████████████| 6.9MB 19.0MB/s 
     |████████████████████████████████| 501kB 45.6MB/s 
     |████████████████████████████████| 102kB 10.3MB/s 


In [2]:
import xarray as xr
import fsspec

ds = xr.open_zarr(fsspec.get_mapper('gcs://weather-bench/region_zarr/'), consolidated=True)

print("Tamanño en GB:", ds.nbytes / 1e9)

Tamanño en GB: 19.324476644


In [3]:
ds

<xarray.Dataset>
Dimensions:    (lat: 12, level: 13, lon: 14, time: 350640)
Coordinates:
  * lat        (lat) float64 -70.31 -64.69 -59.06 ... -19.69 -14.06 -8.438
  * level      (level) int32 50 100 150 200 250 300 400 500 600 700 850 925 1000
  * lon        (lon) float64 247.5 253.1 258.8 264.4 ... 303.8 309.4 315.0 320.6
  * time       (time) datetime64[ns] 1979-01-01 ... 2018-12-31T23:00:00
Data variables:
    lat2d      (lat, lon) float64 dask.array<chunksize=(12, 14), meta=np.ndarray>
    lon2d      (lat, lon) float64 dask.array<chunksize=(12, 14), meta=np.ndarray>
    lsm        (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    orography  (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    pv         (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    q          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    r          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    slt        (lat, lon) float32 dask.array<chunksize=(12, 14), meta=np.ndarray>
    t          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    t2m        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    tcc        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    u10        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    v10        (time, lat, lon) float32 dask.array<chunksize=(43830, 2, 4), meta=np.ndarray>
    vo         (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
    z          (time, level, lat, lon) float32 dask.array<chunksize=(43830, 2, 3, 4), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-03-08 20:51:24 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

# Tree based methods for $Z500$, $T850$ and $T2M$

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from ml_training import *

# from dask_ml.wrappers import ParallelPostFit

## Data Loading

In [7]:
z_500 = ds.z.sel(level=500).drop('level')
t_850 = ds.t.sel(level=850).drop('level')
t2m = ds.t2m

data = xr.merge([z_500, t_850, t2m])

data_train = data.sel(time=slice('2010', '2016'))
data_test = data.sel(time=slice('2017', '2018'))

z_test = load_test_data('z', ds=ds)
t_test = load_test_data('t', ds=ds)
t2m_test = load_test_data('t2m', ds=ds)

test_data = xr.merge([z_test, t_test, t2m_test])

_, nlat, nlon = data_train.z.shape; nlat, nlon

Selecting from ds...
Selecting from ds...


(12, 14)

In [8]:
# Compute normalization statistics
# Let's only take a sample of the time to speed this up.
data_mean = data_train.isel(time=slice(0, None, 10000)).mean().load()
data_std = data_train.isel(time=slice(0, None, 10000)).std().load()


# Normalize datasets
data_train = (data_train - data_mean) / data_std
data_test = (data_test - data_mean) / data_std

### Define combinations of training data

In [9]:
# Inputs and Outputs
experiments = [
    [['z'], ['z']],
    [['t'], ['t']],
    [['z', 't'], ['z', 't']],
    [['t2m'], ['t2m']],
    [['z', 't', 't2m'], ['t2m']],
]

lead_time = 3*24 # Predict 3 days from current time

## Run Experiments

In [10]:
from sklearn.tree import DecisionTreeRegressor

preds_dt_reg = []
models_dt_reg = []

for iter, (i, o) in enumerate(experiments):
    print(f'{iter}: Input variables = {i}; output variables = {o}')
    pred, model = train_regression(DecisionTreeRegressor,
                                   [data_train, data_test, nlat, nlon, data_std, data_mean],
                                   lead_time, 
                                   input_vars=i, 
                                   output_vars=o,
                                   data_subsample=5,
                                   extra_args={'max_depth': 5})
    
    preds_dt_reg.append(pred)
    models_dt_reg.append(model)

    r = compute_weighted_rmse(pred, test_data).compute()
    print('; '.join([f'{v} = {r[v].values}' for v in r]) + '\n')

0: Input variables = ['z']; output variables = ['z']
Train MSE = 0.09194483666311212
Test MSE = 0.09792390299841923
z = 884.2563823840462

1: Input variables = ['t']; output variables = ['t']
Train MSE = 0.12155349657333754
Test MSE = 0.1269337002916634
t = 3.672168092395732

2: Input variables = ['z', 't']; output variables = ['z', 't']
Train MSE = 0.10654099803178393
Test MSE = 0.11049453614805514
z = 877.3803713880076; t = 3.6408053136169216

3: Input variables = ['t2m']; output variables = ['t2m']
Train MSE = 0.05379231407428573
Test MSE = 0.054391407820834736
t2m = 2.485748581044764

4: Input variables = ['z', 't', 't2m']; output variables = ['t2m']
Train MSE = 0.0536680671408368
Test MSE = 0.05451969663333439
t2m = 2.4875064512869165



In [ ]:
from sklearn.ensemble import RandomForestRegressor

preds_rf_reg = []
models_rf_reg = []

for iter, (i, o) in enumerate(experiments):
    print(f'{iter}: Input variables = {i}; output variables = {o}')
    pred, model = train_regression(RandomForestRegressor,
                                   [data_train, data_test, nlat, nlon, data_std, data_mean],
                                   lead_time, 
                                   input_vars=i, 
                                   output_vars=o,
                                   data_subsample=5,
                                   extra_args={'n_jobs': -1, 'verbose': 
                                               3})
    
    preds_rf_reg .append(pred)
    models_rf_reg.append(model)

0: Input variables = ['z']; output variables = ['z']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100building tree 2 of 100

building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100
building tree 17 of 100
building tree 18 of 100
building tree 19 of 100
building tree 20 of 100
building tree 21 of 100
building tree 22 of 100
building tree 23 of 100
building tree 24 of 100
building tree 25 of 100
building tree 26 of 100
building tree 27 of 100
building tree 28 of 100
building tree 29 of 100


[Parallel(n_jobs=-1)]: Done  28 tasks      | elapsed:  6.1min


building tree 30 of 100
building tree 31 of 100
building tree 32 of 100
building tree 33 of 100
building tree 34 of 100
building tree 35 of 100
building tree 36 of 100
building tree 37 of 100
building tree 38 of 100
building tree 39 of 100
building tree 40 of 100
building tree 41 of 100
building tree 42 of 100
building tree 43 of 100
building tree 44 of 100
building tree 45 of 100
building tree 46 of 100
building tree 47 of 100
building tree 48 of 100
building tree 49 of 100
building tree 50 of 100
building tree 51 of 100
building tree 52 of 100
building tree 53 of 100
building tree 54 of 100
building tree 55 of 100
building tree 56 of 100
building tree 57 of 100
building tree 58 of 100
building tree 59 of 100
building tree 60 of 100
building tree 61 of 100
building tree 62 of 100
building tree 63 of 100
building tree 64 of 100
building tree 65 of 100
building tree 66 of 100
building tree 67 of 100
building tree 68 of 100
building tree 69 of 100
building tree 70 of 100
building tree 71

...

Train MSE = 0.004126855522235633
Test MSE = 0.0861123918328185


[Parallel(n_jobs=2)]: Done  28 tasks      | elapsed:    0.2s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    0.9s finished


1: Input variables = ['t']; output variables = ['t']


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.


building tree 1 of 100
building tree 2 of 100
building tree 3 of 100
building tree 4 of 100
building tree 5 of 100
building tree 6 of 100
building tree 7 of 100
building tree 8 of 100
building tree 9 of 100
building tree 10 of 100
building tree 11 of 100
building tree 12 of 100
building tree 13 of 100
building tree 14 of 100
building tree 15 of 100
building tree 16 of 100


In [ ]:
import xgboost as xgb # https://xgboost.readthedocs.io/en/latest/python/python_api.html

preds_xgb_reg= []
models_xgb_reg = []

for iter, (i, o) in enumerate(experiments):
    print(f'{iter}: Input variables = {i}; output variables = {o}')
    pred, model = train_regression(xgb.XGBRFRegressor,
                                   [data_train, data_test, nlat, nlon, data_std, data_mean],
                                   lead_time, 
                                   input_vars=i, 
                                   output_vars=o,
                                   extra_args={})
    
    preds_xgb_reg.append(pred)
    models_xgb_reg.append(model)

    r = compute_weighted_rmse(pred, test_data).compute()
    print('; '.join([f'{v} = {r[v].values}' for v in r]) + '\n')

0: Input variables = ['z']; output variables = ['z']
[20:20:32] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


ValueError: ignored